# 네이버 쇼핑 순위 가져오기

In [1]:
import json
import time
from time import sleep

import urllib
from urllib.request import urlopen

import requests
from bs4 import BeautifulSoup

from selenium import webdriver

import multiprocess as mp
import multiprocessing
from multiprocessing import Process, Manager
from pathos.multiprocessing import ProcessingPool as Pool

#import ggt
import concurrent.futures
import pandas as pd

### 20page로 테스트

In [2]:
keyword = '폼클렌징'
encText = urllib.parse.quote(f'{keyword}') ## 검색하고자 하는 키워드
naver_shop = 'https://search.shopping.naver.com/search/all?'+'frm=NVSCTAB'+'&origQuery='+encText+'&pagingIndex={}'+'&pagingSize=40&productSet=total'+'&query='+encText+'&sort=rel&timestamp=&viewType=list'
urls = [naver_shop.format(i) for i in range(1,21)]

In [3]:
def ggt2(i):
    req = requests.get(urls[i])
    req = req.text
    soup = BeautifulSoup(req, 'html.parser')
    result = soup.find('script', {'type': 'application/json'})
    result = str(result)
    result = result.replace(
        '<script id="__NEXT_DATA__" type="application/json">', '')
    result = result.replace('</script>', '')
    result_json = json.loads(result.strip())
    result_test = (result_json['props']['pageProps']['initialState']
          ['products']['list'][1]['item']['mallName'])
    dic_test[f'{i}페이지 첫번째 값'] = result_test

### 단순

In [4]:
### 단순 계산

start_time = time.time()

#dict 만들기
dic_test = dict()

for i in range(len(urls)):
    ggt2(i)
print(dic_test)
print("--- %s seconds ---" % (time.time() - start_time))

{'0페이지 첫번째 값': '쿠스 공식스토어', '1페이지 첫번째 값': '순수연구소', '2페이지 첫번째 값': '프리메라 브랜드 스토어', '3페이지 첫번째 값': '아모레 헤어바디 공식몰', '4페이지 첫번째 값': '원오세븐', '5페이지 첫번째 값': '토픽피부관리실', '6페이지 첫번째 값': '파파레서피', '7페이지 첫번째 값': '닥터디퍼런트', '8페이지 첫번째 값': '에브리스킨', '9페이지 첫번째 값': '맥시맥시', '10페이지 첫번째 값': '디오비오', '11페이지 첫번째 값': '닥터디퍼런트', '12페이지 첫번째 값': '야다', '13페이지 첫번째 값': '', '14페이지 첫번째 값': '', '15페이지 첫번째 값': '바잉닷컴', '16페이지 첫번째 값': '쿠팡', '17페이지 첫번째 값': '더뷰티1번창고', '18페이지 첫번째 값': '코스제이', '19페이지 첫번째 값': ''}
--- 16.33307981491089 seconds ---


In [5]:
### 값 확인

for i in range(0,len(urls)):
    print(dic_test[f'{i}페이지 첫번째 값'])

쿠스 공식스토어
순수연구소
프리메라 브랜드 스토어
아모레 헤어바디 공식몰
원오세븐
토픽피부관리실
파파레서피
닥터디퍼런트
에브리스킨
맥시맥시
디오비오
닥터디퍼런트
야다


바잉닷컴
쿠팡
더뷰티1번창고
코스제이



### 멀티쓰레드
### 멀티프로세스는 전역변수 해결 못해서 패스.(리눅스 맥에서는 된다고 함. test 해볼것)


In [6]:
### 멀티쓰레드
### max_worker =  로 설정 가능
### 멀티프로세스는 전역변수 해결 못해서 패스.(리눅스 맥에서는 된다고 함. test 해볼것)
### ThreadPoolExecutor --> ProcessPoolExecuto
start_time = time.time()

dic_test = dict()

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(ggt2, [(i) for i in range(0,20)])
    
print(dic_test)
print("--- %s seconds ---" % (time.time() - start_time))


{'5페이지 첫번째 값': '토픽피부관리실', '4페이지 첫번째 값': '원오세븐', '8페이지 첫번째 값': '에브리스킨', '3페이지 첫번째 값': '아모레 헤어바디 공식몰', '9페이지 첫번째 값': '맥시맥시', '6페이지 첫번째 값': '파파레서피', '10페이지 첫번째 값': '디오비오', '2페이지 첫번째 값': '프리메라 브랜드 스토어', '7페이지 첫번째 값': '닥터디퍼런트', '1페이지 첫번째 값': '순수연구소', '11페이지 첫번째 값': '닥터디퍼런트', '13페이지 첫번째 값': '', '0페이지 첫번째 값': '쿠스 공식스토어', '12페이지 첫번째 값': '야다', '14페이지 첫번째 값': '', '15페이지 첫번째 값': '바잉닷컴', '16페이지 첫번째 값': '쿠팡', '19페이지 첫번째 값': '', '18페이지 첫번째 값': '코스제이', '17페이지 첫번째 값': '더뷰티1번창고'}
--- 2.0474205017089844 seconds ---


In [7]:
### 값 확인
for i in range(0,len(urls)):
    print(dic_test[f'{i}페이지 첫번째 값'])

쿠스 공식스토어
순수연구소
프리메라 브랜드 스토어
아모레 헤어바디 공식몰
원오세븐
토픽피부관리실
파파레서피
닥터디퍼런트
에브리스킨
맥시맥시
디오비오
닥터디퍼런트
야다


바잉닷컴
쿠팡
더뷰티1번창고
코스제이



# 블로그 지수 확인

In [2]:
### 아이디입력
idid = 'mamelyoo'

In [3]:
start_time = time.time()

### 크롬 켜기
driver = webdriver.Chrome('C:/chromedriver.exe')
sleep(0.5)

### 주소 접속. 꼭 모바일로 접속하기
driver.get('https://m.blog.naver.com/'+idid)
sleep(0.5)

### 나열식으로 보기
driver.find_element_by_class_name("btn_list").click()
sleep(0.5)

### 파싱
req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

### 크롬 끄기
driver.close()

print("셀레니움 후, 파싱까지만--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

### 닉네임가져오기
nickname = ''
nickname = soup.find(class_='user_name').text

### 제목 가져오기
b = soup.find_all(class_='title ell')
sleep(0.5)

### 제목 tlist랑 공백제거 tlist_no_space(키워드자동완성위함)
tlist = []
tlist_no_space = []

for i in range(0,10):
    tlist.append(b[i].text)
    tlist_no_space.append(b[i].text.replace(" ",""))
########## 링크 linklist ###########
linklist = []
for a in soup.find_all('div',{"class":"postlist"}):
    linklist.append('https://m.blog.naver.com/'+ idid + a['id'].replace('pl_','/'))

셀레니움 후, 파싱까지만--- 5.466476917266846 seconds ---


In [4]:
### 함수 정의 ###
########## 각 글별 자동키워드 가져오기 #########
def find_key_auto(number):
    testlist = []
    testlist.append(nickname)
    testlist.append(str(number) + "번째")
    testlist.append(tlist[number])
    req = requests.get(linklist[number])
    req = req.text
    soup = BeautifulSoup(req, 'html.parser')
    ### 태그가져오기
    if soup.find(class_='post_tag') == None:
        testlist.append(None)
    else:
        tags = soup.find(class_='post_tag').text
        tags = tags.replace("\n","")
        tags = tags.split("#")
        tags = list(filter(None,tags))
        ### 해시 태그와 제목의 중복값 리스트 만들기
        samelist =[]
        for i in tags:
            if i in tlist_no_space[number]:
                samelist.append(i)
            else:
                pass

        ### 가장 긴 키워드를 메인키워드로 하기
        ### 값이 같을시, 먼저 써있는게 우선으로 나옴
        if samelist == []:
            testlist.append('일치 없음')
        else:
            best = 0
            for index in range(len(samelist)):
                if len(samelist[index]) > len(samelist[best]):
                    best = index
            testlist.append(samelist[best])
    testlist.append(linklist[number])
    
    key_auto_list.append(testlist)

In [5]:
### 멀티쓰레드 ###

start_time = time.time()

key_auto_list = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(find_key_auto, [(i) for i in range(0,len(linklist))])

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.0392203330993652 seconds ---


In [6]:
df3 = pd.DataFrame(key_auto_list,columns = ['닉네임','최신순','제목','키워드','URL'])
df3 = df3.sort_values(by=['최신순',],ascending=True)
df3 = df3.reset_index(drop=True)
df3

,닉네임,최신순,제목,키워드,URL
0,마멜유,0번째,"오렌즈 체리문 , 자연광에서 더 맑고 예쁜 키르시 초승달렌즈 신상",오렌즈체리문,https://m.blog.naver.com/mamelyoo/222266309728
1,마멜유,1번째,"립밤추천 , 쟁여두고싶은 토리든 솔리드인 립에센스",솔리드인립에센스,https://m.blog.naver.com/mamelyoo/222265209424
2,마멜유,2번째,"봄저격 아이섀도우 팔레트 , 오프라코스메틱 스윗드림",아이섀도우팔레트,https://m.blog.naver.com/mamelyoo/222264075017
3,마멜유,3번째,올리브영세일 마스카라 득템은 데자뷰마스카라,데자뷰마스카라,https://m.blog.naver.com/mamelyoo/222262111012
4,마멜유,4번째,겨드랑이 색소침착 놉! 블랙샷 바디미백크림 N통째 애용중,겨드랑이색소침착,https://m.blog.naver.com/mamelyoo/222257980425
5,마멜유,5번째,폼클렌징 추천 ! 깨끗하게 화장지우는법,폼클렌징추천,https://m.blog.naver.com/mamelyoo/222254384076
6,마멜유,6번째,"쿠션 추천 , 클리오 킬커버 신상 파운웨어 쿠션 올뉴 전컬러 비교해봄!",클리오킬커버,https://m.blog.naver.com/mamelyoo/222251126391
7,마멜유,7번째,플린 안번지는 펜슬 아이라이너 (Feat.립앤아이리무버 패드),안번지는펜슬아이라이너,https://m.blog.naver.com/mamelyoo/222249672524
8,마멜유,8번째,바디로션 추천 꼼꼼하게 찾은 꿀템!,바디로션추천,https://m.blog.naver.com/mamelyoo/222248467955
9,마멜유,9번째,"이니스프리 그린티 씨드 세럼 , 겨울 수분에센스 찰떡템",이니스프리그린티씨드세럼,https://m.blog.naver.com/mamelyoo/222247845185


In [7]:
auto_choice = list(df3['키워드'])

### 키워드 별 뷰순위 가져오기

In [8]:
### 키워드 입력 ###
### 지금은 오토로 가져온걸로 되게 하기 ###
key_list = auto_choice
key_list

['오렌즈체리문',
 '솔리드인립에센스',
 '아이섀도우팔레트',
 '데자뷰마스카라',
 '겨드랑이색소침착',
 '폼클렌징추천',
 '클리오킬커버',
 '안번지는펜슬아이라이너',
 '바디로션추천',
 '이니스프리그린티씨드세럼']

### 서영님께 요청 ###

### 키워드 리스트를 사용자가 수정할 수 있게 해주기 ###

ex) key_list = ['남자친구생일선물', '디저트', '재생크림'] 등등
- 공란은 None으로 통일하기 > None일시에 검색되지 않도록 설정해둠.
- 검색창에 띄어쓰기하는건 전부다 없애주기 > replace(" ","")

In [9]:
def viewrank(keyword):
    if keyword == None:
        pass
    
    else:
        key_url = 'https://m.search.naver.com/search.naver?where=m_view&query='+ keyword +'&sm=mtb_viw.all&nso=&mode=normal&main_q=&st_coll=&topic_r_cat='
        req = requests.get(key_url)
        req = req.text
        soup = BeautifulSoup(req, 'html.parser')
        ### 첫페이지 40여개 파싱
        search_all = soup.find_all('a',{'class':'api_txt_lines total_tit'})
        ### 랭킹용 숫자
        rank_numb = 0
        ###
        for k in range(0,len(search_all)):
            ### 광고는 지나가기
            if 'https://adcr.naver.com' in search_all[k]['href']:
                pass
            else:
                list_sub = []
                ### 광고 없을 때의 순위 따로 채크
                rank_numb = rank_numb + 1
                list_sub.append(rank_numb)
                ### 제목
                title = search_all[k].text
                list_sub.append(keyword)
                list_sub.append(title)
                list_sub.append(search_all[k]['href'])

                list_main.append(list_sub)

In [10]:
### 멀티쓰레드

start_time = time.time()

list_main = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(viewrank, key_list)

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.8151443004608154 seconds ---


In [11]:
print(list_main)

[[1, '오렌즈체리문', '신제품 오렌즈 x 키르시 콜라보 체리문 초승달렌즈 은은한 봄메이크업에 딱', 'https://m.blog.naver.com/lqud/222264723034'], [2, '오렌즈체리문', '오렌즈 체리문 , 자연광에서 더 맑고 예쁜 키르시 초승달렌즈 신상', 'https://m.blog.naver.com/mamelyoo/222266309728'], [3, '오렌즈체리문', '오렌즈 체리문 브라운과 그레이, 키르시 콜라보로 나온 신제품 초승달렌즈 심쿵!', 'https://m.blog.naver.com/haha1126gk/222266398147'], [4, '오렌즈체리문', '오렌즈 체리문 신제품 초승달렌즈! 키르시 콜라보 뷰티키트까지 만나봄', 'https://m.blog.naver.com/yangeunb/222262941912'], [5, '오렌즈체리문', '오렌즈X키르시 신제품 영롱보스 체리문 초승달렌즈 이렇게 예쁠수가,,♥', 'https://m.blog.naver.com/fishau/222265405325'], [6, '오렌즈체리문', '*체리문한달용렌즈출시*오렌즈/오렌즈해운대점/해운대오렌즈/장산역오렌즈/장산오렌즈/체리문/부산오렌즈...', 'https://m.blog.naver.com/nofear1343/222266218999'], [7, '오렌즈체리문', "[오렌즈 반월당점] 한달용 신상 렌즈 NEW '체리문' 출시", 'https://m.blog.naver.com/olens5522/222265297636'], [8, '오렌즈체리문', '[북수원오렌즈] 오렌즈 X 키르시 :: 오렌즈체리문 초승달 패턴 신제품 출시/ 키르시 콜라보 뷰티키트까지', 'https://m.blog.naver.com/olens_buksuwon/222266188073'], [9, '오렌즈체리문', '오렌즈 신상 렌즈 두둥 등장!( ͝° ͜ʖ͡°) /#체리문 #오렌즈체리문 #오렌즈신상 #신상렌즈', 'https://m.blog.naver.com/olen

In [12]:
df2 = pd.DataFrame(list_main,columns = ['랭킹','키워드','제목','URL'])
df2 = df2.sort_values(by=['키워드','랭킹',],ascending=True)
df2 = df2.reset_index(drop=True)
df2

,랭킹,키워드,제목,URL
0,1,겨드랑이색소침착,겨드랑이 색소침착 쉽게 해결해봐요,https://m.blog.naver.com/nana6564/222257726942
1,2,겨드랑이색소침착,겨드랑이 색소침착 쉽게 없애,https://m.blog.naver.com/pseuryvs/222220840954
2,3,겨드랑이색소침착,겨드랑이 색소침착 놉! 블랙샷 바디미백크림 N통째 애용중,https://m.blog.naver.com/mamelyoo/222257980425
3,4,겨드랑이색소침착,겨드랑이 색소침착 크림 환해지는 노하우,https://m.blog.naver.com/typep/222183082458
4,5,겨드랑이색소침착,겨드랑이 색소침착 신경쓰이더라구요,https://m.blog.naver.com/dyn240/222250600188
...,...,...,...,...
292,26,폼클렌징추천,올리브영 폼클렌징 추천 애정템 발견,https://m.blog.naver.com/asm9911/222246436753
293,27,폼클렌징추천,폼클렌징 추천좀요~,https://m.cafe.naver.com/miznett/2654653
294,28,폼클렌징추천,폼클렌징 추천 세정력 찐이네!,https://m.blog.naver.com/reireyofficial/222197...
295,29,폼클렌징추천,턱 트러블 클렌징폼,https://m.cafe.naver.com/cosmania/31793194


In [13]:
df3

,닉네임,최신순,제목,키워드,URL
0,마멜유,0번째,"오렌즈 체리문 , 자연광에서 더 맑고 예쁜 키르시 초승달렌즈 신상",오렌즈체리문,https://m.blog.naver.com/mamelyoo/222266309728
1,마멜유,1번째,"립밤추천 , 쟁여두고싶은 토리든 솔리드인 립에센스",솔리드인립에센스,https://m.blog.naver.com/mamelyoo/222265209424
2,마멜유,2번째,"봄저격 아이섀도우 팔레트 , 오프라코스메틱 스윗드림",아이섀도우팔레트,https://m.blog.naver.com/mamelyoo/222264075017
3,마멜유,3번째,올리브영세일 마스카라 득템은 데자뷰마스카라,데자뷰마스카라,https://m.blog.naver.com/mamelyoo/222262111012
4,마멜유,4번째,겨드랑이 색소침착 놉! 블랙샷 바디미백크림 N통째 애용중,겨드랑이색소침착,https://m.blog.naver.com/mamelyoo/222257980425
5,마멜유,5번째,폼클렌징 추천 ! 깨끗하게 화장지우는법,폼클렌징추천,https://m.blog.naver.com/mamelyoo/222254384076
6,마멜유,6번째,"쿠션 추천 , 클리오 킬커버 신상 파운웨어 쿠션 올뉴 전컬러 비교해봄!",클리오킬커버,https://m.blog.naver.com/mamelyoo/222251126391
7,마멜유,7번째,플린 안번지는 펜슬 아이라이너 (Feat.립앤아이리무버 패드),안번지는펜슬아이라이너,https://m.blog.naver.com/mamelyoo/222249672524
8,마멜유,8번째,바디로션 추천 꼼꼼하게 찾은 꿀템!,바디로션추천,https://m.blog.naver.com/mamelyoo/222248467955
9,마멜유,9번째,"이니스프리 그린티 씨드 세럼 , 겨울 수분에센스 찰떡템",이니스프리그린티씨드세럼,https://m.blog.naver.com/mamelyoo/222247845185


In [14]:
ranking_list = []

for i in range(0,len(list_main)):
    testlist=[]
    if df2['URL'][i] in list(df3['URL']) :
        testlist.append(int(df2['랭킹'][i]))
        testlist.append(str(df2['URL'][i]))
        ranking_list.append(testlist)
    else:
        pass

In [15]:
df4 = pd.DataFrame(ranking_list,columns = ['랭킹','URL'])
df4 = df4.sort_values(by=['랭킹','URL',],ascending=True)
df4 = df4.reset_index(drop=True)
df4

,랭킹,URL
0,1,https://m.blog.naver.com/mamelyoo/222249672524
1,1,https://m.blog.naver.com/mamelyoo/222264075017
2,2,https://m.blog.naver.com/mamelyoo/222262111012
3,2,https://m.blog.naver.com/mamelyoo/222266309728
4,3,https://m.blog.naver.com/mamelyoo/222257980425
5,7,https://m.blog.naver.com/mamelyoo/222265209424
6,9,https://m.blog.naver.com/mamelyoo/222251126391
7,12,https://m.blog.naver.com/mamelyoo/222254384076
8,19,https://m.blog.naver.com/mamelyoo/222247845185
9,27,https://m.blog.naver.com/mamelyoo/222248467955


In [16]:
data = pd.merge(df3,df4,on="URL")
data

,닉네임,최신순,제목,키워드,URL,랭킹
0,마멜유,0번째,"오렌즈 체리문 , 자연광에서 더 맑고 예쁜 키르시 초승달렌즈 신상",오렌즈체리문,https://m.blog.naver.com/mamelyoo/222266309728,2
1,마멜유,1번째,"립밤추천 , 쟁여두고싶은 토리든 솔리드인 립에센스",솔리드인립에센스,https://m.blog.naver.com/mamelyoo/222265209424,7
2,마멜유,2번째,"봄저격 아이섀도우 팔레트 , 오프라코스메틱 스윗드림",아이섀도우팔레트,https://m.blog.naver.com/mamelyoo/222264075017,1
3,마멜유,3번째,올리브영세일 마스카라 득템은 데자뷰마스카라,데자뷰마스카라,https://m.blog.naver.com/mamelyoo/222262111012,2
4,마멜유,4번째,겨드랑이 색소침착 놉! 블랙샷 바디미백크림 N통째 애용중,겨드랑이색소침착,https://m.blog.naver.com/mamelyoo/222257980425,3
5,마멜유,5번째,폼클렌징 추천 ! 깨끗하게 화장지우는법,폼클렌징추천,https://m.blog.naver.com/mamelyoo/222254384076,12
6,마멜유,6번째,"쿠션 추천 , 클리오 킬커버 신상 파운웨어 쿠션 올뉴 전컬러 비교해봄!",클리오킬커버,https://m.blog.naver.com/mamelyoo/222251126391,9
7,마멜유,7번째,플린 안번지는 펜슬 아이라이너 (Feat.립앤아이리무버 패드),안번지는펜슬아이라이너,https://m.blog.naver.com/mamelyoo/222249672524,1
8,마멜유,8번째,바디로션 추천 꼼꼼하게 찾은 꿀템!,바디로션추천,https://m.blog.naver.com/mamelyoo/222248467955,27
9,마멜유,9번째,"이니스프리 그린티 씨드 세럼 , 겨울 수분에센스 찰떡템",이니스프리그린티씨드세럼,https://m.blog.naver.com/mamelyoo/222247845185,19


In [17]:
a = [4,2,3,1]
a=sorted(a)
a

[1, 2, 3, 4]